In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import glob
from PIL import Image
import numpy as np 
from torchvision.io import read_image
import cv2
from torchvision.utils import save_image
import os
from torch.utils import data
import cv2
import math
import torchvision

In [2]:
class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))

    def forward(self, x):
        s = self.conv(x)
        p = self.pool(s)
        return s, p

class attention_gate(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.Wg = nn.Sequential(
            nn.Conv2d(in_c[0], out_c, kernel_size=1, padding=0),
            nn.BatchNorm2d(out_c)
        )
        self.Ws = nn.Sequential(
            nn.Conv2d(in_c[1], out_c, kernel_size=1, padding=0),
            nn.BatchNorm2d(out_c)
        )
        self.relu = nn.ReLU(inplace=True)
        self.output = nn.Sequential(
            nn.Conv2d(out_c, out_c, kernel_size=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, g, s):
        Wg = self.Wg(g)
        Ws = self.Ws(s)
        out = self.relu(Wg + Ws)
        out = self.output(out)
        return out * s

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.ag = attention_gate(in_c, out_c)
        self.c1 = conv_block(in_c[0]+out_c, out_c)

    def forward(self, x, s):
        x = self.up(x)
        s = self.ag(x, s)
        x = torch.cat([x, s], axis=1)
        x = self.c1(x)
        return x

class attention_unet(nn.Module):
    def __init__(self):
        super().__init__()

        self.e1 = encoder_block(3, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)

        self.b1 = conv_block(256, 512)

        self.d1 = decoder_block([512, 256], 256)
        self.d2 = decoder_block([256, 128], 128)
        self.d3 = decoder_block([128, 64], 64)

        self.output = nn.Conv2d(64, 3, kernel_size=1, padding=0)

    def forward(self, x):
        s1, p1 = self.e1(x)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)

        b1 = self.b1(p3)

        d1 = self.d1(b1, s3)
        d2 = self.d2(d1, s2)
        d3 = self.d3(d2, s1)

        output = self.output(d3)
        return output

In [14]:
transform = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.ToTensor()
])

class CreateDataset(data.Dataset):
    def __init__(self):
        self.inputs = sorted(glob.glob('C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/train_in/*.jpg'))[:]
        self.targets = sorted(glob.glob('C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/train_out/*.jpg'))[:]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,index: int):

        input_path = self.inputs[index]
        output_path = self.targets[index]
        #print(input_path)
        #print(output_path)
        assert os.path.basename(input_path) == os.path.basename(output_path)

        input = transform(Image.open(input_path)).cuda()
        target = transform(Image.open(output_path)).cuda()

        return input,target

dataset = CreateDataset()
dataloader = data.DataLoader(dataset=dataset,batch_size=1,shuffle=True)

In [15]:
transform = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.ToTensor()
])

class CreateDataset(data.Dataset):
    def __init__(self):
        self.inputs = sorted(glob.glob('C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_in/*.jpg'))[:]
        self.targets = sorted(glob.glob('C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_out/*.jpg'))[:]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,index: int):

        input_path = self.inputs[index]
        output_path = self.targets[index]
        print(input_path)
        print(output_path)
        assert os.path.basename(input_path) == os.path.basename(output_path)

        input = transform(Image.open(input_path)).cuda()
        target = transform(Image.open(output_path)).cuda()

        return input,target

test_dataset = CreateDataset()
test_dataloader = data.DataLoader(dataset=test_dataset,batch_size=1,shuffle=True)

In [17]:
model = attention_unet()
model.load_state_dict(torch.load(f='./final_model.pth'))
model.cuda()

attention_unet(
  (e1): encoder_block(
    (conv): conv_block(
      (conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (e2): encoder_block(
    (conv): conv_block(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(128,

In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.001/100)
criterion = nn.MSELoss()

In [7]:
def jaccard_index(pred_mask, true_mask):
    intersection = torch.logical_and(pred_mask, true_mask).sum()
    union = torch.logical_or(pred_mask, true_mask).sum()
    jaccard = intersection.float() / union.float()
    return jaccard.item()

# Example usage:
# pred_mask = torch.tensor([[1, 0, 1],
#                           [0, 1, 1]])
# true_mask = torch.tensor([[1, 1, 0],
#                           [1, 1, 1]])

# iou = jaccard_index(pred_mask, true_mask)


In [8]:
model.train()

for epoch in range(1) :
    
    totalLoss = 0
    acc = 0

    for idx,(inputs,targets) in enumerate(dataloader):

        print(inputs.shape)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs,targets)

        loss.backward()

        optimizer.step()
                
        acc += jaccard_index(outputs,targets)

        totalLoss += loss.item()

        print("Epoch: " + str(epoch) + " Idx : " + str(idx))
        print("Loss: {:.4f}".format(loss.item()))

    print(totalLoss)
    print("IOU : ")
    print(acc/400)

torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 0
Loss: 0.0006
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 1
Loss: 0.0012
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 2
Loss: 0.0015
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 3
Loss: 0.0012
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 4
Loss: 0.0022
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 5
Loss: 0.0012
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 6
Loss: 0.0005
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 7
Loss: 0.0003
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 8
Loss: 0.0013
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 9
Loss: 0.0018
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 10
Loss: 0.0010
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 11
Loss: 0.0006
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 12
Loss: 0.0020
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 13
Loss: 0.0014
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 14
Loss: 0.0009
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 15
Loss: 0.0013
torch.Size([1, 3, 256, 256])
Epoch: 0 Idx : 16
Los

In [12]:
torch.save(obj=model.state_dict(),f='./final_model.pth')

In [9]:
def calculate_dice_coefficient(predictions, targets, smooth=1e-5):
    """
    Calculates the Dice coefficient between predicted and target labels.
    Assumes predictions and targets are torch tensors of the same shape.
    """
    # Flatten the predictions and targets tensors
    predictions_flat = predictions.view(-1)
    targets_flat = targets.view(-1)

    # Calculate the intersection and union
    intersection = torch.sum(predictions_flat * targets_flat)
    union = torch.sum(predictions_flat) + torch.sum(targets_flat)

    # Calculate the Dice coefficient
    dice_coefficient = (2 * intersection + smooth) / (union + smooth)

    return dice_coefficient

def calculate_precision(predictions, targets, smooth=1e-5):
    """
    Calculates the precision between predicted and target labels.
    Assumes predictions and targets are torch tensors of the same shape.
    """
    # Flatten the predictions and targets tensors
    predictions_flat = predictions.view(-1)
    targets_flat = targets.view(-1)

    # Calculate true positives and false positives
    true_positives = torch.sum(predictions_flat * targets_flat)
    false_positives = torch.sum(predictions_flat) - true_positives

    # Calculate precision
    precision = (true_positives + smooth) / (true_positives + false_positives + smooth)

    return precision


def calculate_recall(predictions, targets, smooth=1e-5):
    """
    Calculates the recall between predicted and target labels.
    Assumes predictions and targets are torch tensors of the same shape.
    """
    # Flatten the predictions and targets tensors
    predictions_flat = predictions.view(-1)
    targets_flat = targets.view(-1)

    # Calculate true positives and false negatives
    true_positives = torch.sum(predictions_flat * targets_flat)
    false_negatives = torch.sum(targets_flat) - true_positives

    # Calculate recall
    recall = (true_positives + smooth) / (true_positives + false_negatives + smooth)

    return recall

In [11]:
model.eval()


    
totalLoss = 0
iou = 0
dice_coeff = 0
prec = 0
recall = 0

for idx,(inputs,targets) in enumerate(test_dataloader):
    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs,targets)

        #loss.backward()

        #optimizer.step()
                
    print(jaccard_index(outputs,targets))
    print(calculate_dice_coefficient(outputs, targets))
    print(calculate_precision(outputs,targets))
    print(calculate_recall(outputs,targets))
    iou += jaccard_index(outputs,targets)
    dice_coeff += calculate_dice_coefficient(outputs, targets)
    prec += calculate_precision(outputs,targets)
    recall += calculate_recall(outputs,targets)
    totalLoss += loss.item()

    loss.backward()
    optimizer.step()

       
print("Total Loss : ")
print(totalLoss)
print("Avg IOU : ")
print(iou/100)
print("Avg Dice Coefficient : ")
print(dice_coeff/100)
print("Avg Precision : ")
print(prec/100)
print("Avg Recall : ")
print(recall/100)

C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_in\59.jpg
C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_out\59.jpg
0.9971873164176941
tensor(0.5402, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.5248, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.5566, device='cuda:0', grad_fn=<DivBackward0>)
C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_in\300.jpg
C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_out\300.jpg
0.978179931640625
tensor(0.4490, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.4595, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.4390, device='cuda:0', grad_fn=<DivBackward0>)
C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_in\489.jpg
C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_out\489.jpg
0.9695332646369934
tensor(0.4916, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.4842, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.4991, device='cuda:0', grad_fn=<DivBackward0>)
C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/

In [25]:
model.eval()

t = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.ToTensor()
])

with torch.no_grad() :
    test_input = t(Image.open('C:/Users/sahil/OneDrive/Desktop/IITP/New_Dataset/test_in/499.jpg'))
    temp = test_input
    test_input = torch.unsqueeze(test_input,dim=0).cuda()
    test_output = model(test_input)
    test_output = torch.squeeze(test_output)
    save_image(test_output,'output1.jpg')
    
model.cuda()

attention_unet(
  (e1): encoder_block(
    (conv): conv_block(
      (conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (e2): encoder_block(
    (conv): conv_block(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(128,